In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from datetime import datetime, timedelta
import datetime as dt
from django.utils import timezone
from bookings.models import Room, RoomType, Staff, Booking

from django.db.models import *
from django.db.models.functions import *

# 1. CRUD operations

1.1 ให้เพิ่มห้อง (`Room`) โดยมีข้อมูลดังนี้ (0.5 คะแนน)

- number: 203
- name: Laboratory A
- capacity: 50
- description: Laboratory for Web Development
- room_types: Lecture, Office

In [4]:
# CODE HERE
lecture = RoomType.objects.get(name="Lecture")
office = RoomType.objects.get(name="Office")
room = Room.objects.create(number="203", name="Laboratory A", capacity=50, description="Laboratory for Web Development")
room.room_types.add(lecture, office)

In [5]:
# CHECK RESULTS (ห้ามลบ/แก้ไข)
r = Room.objects.get(number="203")
for t in r.room_types.all():
    print(f"ROOM: {r}; ROOM TYPE: {t.name}")

ROOM: Room 203: Laboratory A; ROOM TYPE: Lecture
ROOM: Room 203: Laboratory A; ROOM TYPE: Office


1.2 เนื่องจากนโยบายการจองคือห้ามจองห้องเกิน 1 ชั่วโมง ดังนั้นให้ค้นหาการจองห้อง (`Booking`) ที่มีระยะเวลามากกว่า 1 ชั่วโมงและทำการลบออกทั้งหมด (1 คะแนน)

**Hint: ให้ annotate() คำนวณระยะเวลาการจอง และนำระยะเวลาการจองมา filter() ดังตัวอย่าง**

```python
from datetime import timedelta

queryset.filter(
    duration__gt=timedelta(hours=1)
)
```

In [22]:
# CODE HERE - Filter การจองที่เกิน 1 ชม.
Booking.objects.annotate(duration=F("end_time")-F("start_time")).filter(duration__gt=timedelta(hours=1))

<QuerySet [<Booking: Booking by John Doe for Room 101: Main Conference Room from 2024-08-18 02:00:00+00:00 to 2024-08-18 04:00:00+00:00>, <Booking: Booking by Jane Smith for Room 202: Lecture Hall A from 2024-08-19 05:00:00+00:00 to 2024-08-19 07:00:00+00:00>, <Booking: Booking by John Doe for Room 101: Main Conference Room from 2024-08-19 03:00:00+00:00 to 2024-08-19 05:00:00+00:00>, <Booking: Booking by Emily Davis for Room 303: Executive Office from 2024-08-22 07:00:00+00:00 to 2024-08-22 08:30:00+00:00>, <Booking: Booking by John Doe for Room 303: Executive Office from 2024-08-24 04:00:00+00:00 to 2024-08-24 06:00:00+00:00>, <Booking: Booking by Emily Davis for Room 202: Lecture Hall A from 2024-08-26 04:00:00+00:00 to 2024-08-26 05:30:00+00:00>, <Booking: Booking by John Doe for Room 202: Lecture Hall A from 2024-08-30 08:00:00+00:00 to 2024-08-30 10:00:00+00:00>, <Booking: Booking by Emily Davis for Room 303: Executive Office from 2024-08-30 05:00:00+00:00 to 2024-08-30 06:30:00+

In [9]:
# CHECK RESULTS - จำนวนการจองทั้งหมด (ห้ามลบ/แก้ไข)
Booking.objects.count()

33

In [23]:
# CODE HERE - ลบการจองที่เกิน 1 ชม.
Booking.objects.annotate(duration=F("end_time")-F("start_time")).filter(duration__gt=timedelta(hours=1)).delete()

(13, {'bookings.Booking': 13})

In [24]:
# CHECK RESULTS - จำนวนการจองทั้งหมดหลังการลบ (ห้ามลบ/แก้ไข)
Booking.objects.count()

20

1.3 ทำการเพิ่มการจอง (`Booking`) ซึ่งมีรายละเอียดดังนี้ (1 คะแนน)

- staff: "Jane Smith"
- room: "Lecture Hall A"
- start_time: 26/08/2024 13:00
- end_time: 26/08/2024 14:00
- purpose: "To teach subject: IT Project Management"

In [29]:
# CODE HERE
staff = Staff.objects.get(name="Jane Smith")
room = Room.objects.get(name="Lecture Hall A")
Booking.objects.create(staff=staff, room=room, start_time=datetime(2024, 8, 26, 13), end_time=datetime(2024, 8, 26, 14), purpose="To teach subject: IT Project Management")

C:\Users\LAB205_xx\Desktop\New folder\myvenv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Booking.start_time received a naive datetime (2024-08-26 13:00:00) while time zone support is active.
  warnings.warn(
C:\Users\LAB205_xx\Desktop\New folder\myvenv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Booking.end_time received a naive datetime (2024-08-26 14:00:00) while time zone support is active.
  warnings.warn(


<Booking: Booking by Jane Smith for Room 202: Lecture Hall A from 2024-08-26 13:00:00 to 2024-08-26 14:00:00>

In [30]:
# CHECK RESULTS (ห้ามลบ/แก้ไข)
booking = Booking.objects.last()
print(f"STAFF: {booking.staff}, ROOM: {booking.room}, START: {timezone.localtime(booking.start_time)} - {timezone.localtime(booking.end_time)}")

STAFF: Jane Smith, ROOM: Room 202: Lecture Hall A, START: 2024-08-26 13:00:00+00:00 - 2024-08-26 14:00:00+00:00


1.4 จากข้อ 1.3 คุณ "Jane Smith" จองห้องผิดจริงๆ เขาอยากจะจองเป็น "Laboratory A" (ที่สร้างในข้อ 1.1) (0.5 คะแนน)

In [35]:
# CODE HERE
booking = Booking.objects.last()
room = Room.objects.get(name="Laboratory A")
booking.room = room
booking.save()

In [36]:
# CHECK RESULTS (ห้ามลบ/แก้ไข)
booking = Booking.objects.last()
print(f"STAFF: {booking.staff}, ROOM: {booking.room}, START: {timezone.localtime(booking.start_time)} - {timezone.localtime(booking.end_time)}")

STAFF: Jane Smith, ROOM: Room 203: Laboratory A, START: 2024-08-26 13:00:00+00:00 - 2024-08-26 14:00:00+00:00


# 2. Making Queries

**ให้ทำการ reset DB และ import ข้อมูลใน bookings.sql เข้าไปใหม่**

2.1 ให้หาจำนวนการจองที่จองในช่วงวันที่ (`Booking`) ในช่วงวันที่ 2024-08-18 ถึง 2024-08-25 ที่จองห้องในประเภท (`RoomType`) "Conference" (1 คะแนน)

**Expected Output**

จะต้องมี 7 รายการ

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [9]:
# CODE HERE
Booking.objects.filter(start_time__range=(datetime(2024, 8, 18), datetime(2024, 8, 25)), room__room_types__name="Conference").count()

7

2.2 ให้หาจำนวนการจองของห้องแต่ละห้อง โดยแสดงผลดังนี้ - เรียงจากจำนวนการจอง มาก ไป น้อย (1 คะแนน)

**Expected Output**

```
ROOM: Room 101: Main Conference Room, BOOKINGS: 12
ROOM: Room 303: Executive Office, BOOKINGS: 12
ROOM: Room 202: Lecture Hall A, BOOKINGS: 9
```

In [13]:
# CODE HERE
for i in Room.objects.values("name", "number").annotate(bookings=Count("booking")).order_by("-bookings", "number"):
    print("ROOM: Room %s: %s, BOOKINGS: %d"%(i['number'], i['name'], i['bookings']))

ROOM: Room 101: Main Conference Room, BOOKINGS: 12
ROOM: Room 303: Executive Office, BOOKINGS: 12
ROOM: Room 202: Lecture Hall A, BOOKINGS: 9


2.3 ในช่วงวันที่ 2024-08-18 ถึง 2024-08-25 ให้หาว่าพนักงานแต่ละคนมีการจองห้องไหนบ้าง โดยแสดงผลดังนี้ (2 คะแนน)

**Expected Output**

```
STAFF: John Doe, BOOKED ROOMS: Main Conference Room, Executive Office
STAFF: Jane Smith, BOOKED ROOMS: Main Conference Room, Executive Office, Lecture Hall A
STAFF: Emily Davis, BOOKED ROOMS: Lecture Hall A, Executive Office
```

In [39]:
# CODE HERE
for i in Booking.objects.filter(start_time__range=(datetime(2024, 8, 18), datetime(2024, 8, 25))).distinct("staff__name").order_by("-staff__name"):
    print("STAFF: %s, BOOKED ROOMS: "%i.staff.name, end="")
    print(*Booking.objects.filter(staff__name=i.staff.name, start_time__range=(datetime(2024, 8, 18), datetime(2024, 8, 25))).values_list("room__name", flat=True).order_by("-room__name").distinct(), sep=", ")

STAFF: John Doe, BOOKED ROOMS: Main Conference Room, Executive Office
STAFF: Jane Smith, BOOKED ROOMS: Main Conference Room, Lecture Hall A, Executive Office
STAFF: Emily Davis, BOOKED ROOMS: Lecture Hall A, Executive Office


2.4 ให้หาว่าการจองห้องในแต่ละวันมีค่าความจุ (ดูจาก `Room.capacity`) โดยรวมเท่าไหร่ เรียงลำดับความจุรวมจากมากไปน้อย (2 คะแนน)

**Expected Output**

```
DATE: 2024-08-26, SUM CAPACITY: 155
DATE: 2024-08-24, SUM CAPACITY: 155
DATE: 2024-08-30, SUM CAPACITY: 155
DATE: 2024-08-19, SUM CAPACITY: 155
DATE: 2024-09-02, SUM CAPACITY: 100
DATE: 2024-09-06, SUM CAPACITY: 100
DATE: 2024-09-12, SUM CAPACITY: 100
DATE: 2024-09-07, SUM CAPACITY: 100
DATE: 2024-09-13, SUM CAPACITY: 100
DATE: 2024-09-14, SUM CAPACITY: 55
DATE: 2024-08-18, SUM CAPACITY: 55
DATE: 2024-08-28, SUM CAPACITY: 55
DATE: 2024-09-05, SUM CAPACITY: 50
DATE: 2024-09-08, SUM CAPACITY: 50
DATE: 2024-09-01, SUM CAPACITY: 50
DATE: 2024-09-10, SUM CAPACITY: 50
DATE: 2024-09-03, SUM CAPACITY: 50
DATE: 2024-09-11, SUM CAPACITY: 5
DATE: 2024-08-27, SUM CAPACITY: 5
DATE: 2024-09-09, SUM CAPACITY: 5
DATE: 2024-09-04, SUM CAPACITY: 5
DATE: 2024-08-22, SUM CAPACITY: 5
```

In [3]:
# CODE HERE
for i in Booking.objects.values("start_time__date").annotate(total=Sum("room__capacity")).order_by("-total"):
    print("DATE: %s, SUM CAPACITY: %d"%(i['start_time__date'], i['total']))

DATE: 2024-08-30, SUM CAPACITY: 155
DATE: 2024-08-24, SUM CAPACITY: 155
DATE: 2024-08-19, SUM CAPACITY: 155
DATE: 2024-08-26, SUM CAPACITY: 155
DATE: 2024-09-02, SUM CAPACITY: 100
DATE: 2024-09-06, SUM CAPACITY: 100
DATE: 2024-09-07, SUM CAPACITY: 100
DATE: 2024-09-12, SUM CAPACITY: 100
DATE: 2024-09-13, SUM CAPACITY: 100
DATE: 2024-08-28, SUM CAPACITY: 55
DATE: 2024-09-14, SUM CAPACITY: 55
DATE: 2024-08-18, SUM CAPACITY: 55
DATE: 2024-09-10, SUM CAPACITY: 50
DATE: 2024-09-08, SUM CAPACITY: 50
DATE: 2024-09-01, SUM CAPACITY: 50
DATE: 2024-09-03, SUM CAPACITY: 50
DATE: 2024-09-05, SUM CAPACITY: 50
DATE: 2024-09-09, SUM CAPACITY: 5
DATE: 2024-09-04, SUM CAPACITY: 5
DATE: 2024-08-22, SUM CAPACITY: 5
DATE: 2024-08-27, SUM CAPACITY: 5
DATE: 2024-09-11, SUM CAPACITY: 5


2.5 จงหาการจองที่มีวัตถุประสงค์ (`Booking.purpose`) ที่มีความยาวตัวอักษรมากกว่า 20 ตัวอักษร หรือ จะต้องใช้ห้องที่มีความจุมากกว่าหรือเท่ากับ 100 (1 คะแนน)

**Expected Output**

```
START DATE: 2024-08-18, ROOM: Room 101: Main Conference Room, PURPOSE LENGTH: 21, ROOM CAP: 50
START DATE: 2024-08-19, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 19, ROOM CAP: 100
START DATE: 2024-08-19, ROOM: Room 303: Executive Office, PURPOSE LENGTH: 22, ROOM CAP: 5
START DATE: 2024-08-24, ROOM: Room 303: Executive Office, PURPOSE LENGTH: 25, ROOM CAP: 5
START DATE: 2024-08-24, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 15, ROOM CAP: 100
START DATE: 2024-08-26, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 12, ROOM CAP: 100
START DATE: 2024-08-27, ROOM: Room 303: Executive Office, PURPOSE LENGTH: 21, ROOM CAP: 5
START DATE: 2024-08-30, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 19, ROOM CAP: 100
START DATE: 2024-08-30, ROOM: Room 303: Executive Office, PURPOSE LENGTH: 23, ROOM CAP: 5
START DATE: 2024-09-02, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 21, ROOM CAP: 100
START DATE: 2024-09-03, ROOM: Room 101: Main Conference Room, PURPOSE LENGTH: 24, ROOM CAP: 50
START DATE: 2024-09-05, ROOM: Room 101: Main Conference Room, PURPOSE LENGTH: 23, ROOM CAP: 50
START DATE: 2024-09-06, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 22, ROOM CAP: 100
START DATE: 2024-09-07, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 18, ROOM CAP: 100
START DATE: 2024-09-10, ROOM: Room 101: Main Conference Room, PURPOSE LENGTH: 22, ROOM CAP: 50
START DATE: 2024-09-12, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 22, ROOM CAP: 100
START DATE: 2024-09-13, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 14, ROOM CAP: 100
START DATE: 2024-09-14, ROOM: Room 101: Main Conference Room, PURPOSE LENGTH: 21, ROOM CAP: 50
```

In [133]:
# CODE HERE
for i in Booking.objects.annotate(purpose_len=Length("purpose")).filter(Q(purpose_len__gt=20)| Q(room__capacity__gte=100)):
    print("START DATE: %s, ROOM: Room %s: %s, PURPOSE LENGTH: %d, ROOM CAP: %d"%(i.start_time.date(), i.room.number, i.room.name, i.purpose_len, i.room.capacity))

START DATE: 2024-08-18, ROOM: Room 101: Main Conference Room, PURPOSE LENGTH: 21, ROOM CAP: 50
START DATE: 2024-08-19, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 19, ROOM CAP: 100
START DATE: 2024-08-19, ROOM: Room 303: Executive Office, PURPOSE LENGTH: 22, ROOM CAP: 5
START DATE: 2024-08-24, ROOM: Room 303: Executive Office, PURPOSE LENGTH: 25, ROOM CAP: 5
START DATE: 2024-08-24, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 15, ROOM CAP: 100
START DATE: 2024-08-26, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 12, ROOM CAP: 100
START DATE: 2024-08-27, ROOM: Room 303: Executive Office, PURPOSE LENGTH: 21, ROOM CAP: 5
START DATE: 2024-08-30, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 19, ROOM CAP: 100
START DATE: 2024-08-30, ROOM: Room 303: Executive Office, PURPOSE LENGTH: 23, ROOM CAP: 5
START DATE: 2024-09-02, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 21, ROOM CAP: 100
START DATE: 2024-09-03, ROOM: Room 101: Main Conference Room, PURPOSE LENGTH: 24, ROOM CAP: 50
